In [1]:
import numpy as np
import pandas as pd
import sqlite3
import json
import operator
import nltk
import string
from nltk.corpus import stopwords
import time

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
sqlite_file = 'Patent-Project-Rev1.sqlite'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

In [5]:
c.execute("SELECT patent_number, no_patents_citeby FROM Patents WHERE patent_type = 'design' OR patent_type = 'plant' OR patent_type = 'utility' AND patent_abstract IS NOT NULL")
patents_mcited = []
patents_citedby = []
while True:
    getpatent = c.fetchone()
    if getpatent == None:
        break
    patents_mcited.append(getpatent[0])
    patents_citedby.append(getpatent[1])

In [13]:
print len(patents_mcited)
print len(patents_citedby)

837747
837747
0


In [32]:
thres = 100
patents_most_cited = []
for i in range(len(patents_citedby)):
    if patents_citedby[i] >= thres:
        patents_most_cited.append(patents_mcited[i])

In [33]:
len(patents_most_cited)

2171

In [35]:
patents_most_cited[0:10]

[u'8925782',
 u'8925788',
 u'8926598',
 u'8931682',
 u'8936614',
 u'8939343',
 u'8939344',
 u'8960520',
 u'8960521',
 u'8967443']

In [36]:
patents_mcited = []
patents_citedby = []

In [47]:
c.execute("SELECT patent_number, patent_title, patent_abstract FROM Patents WHERE patent_type = 'design' OR patent_type = 'plant' OR patent_type = 'utility' AND patent_abstract IS NOT NULL AND patent_title IS NOT NULL")
patent_list = []
abstract_title_list = []
while True:
    getpatent = c.fetchone()
    if getpatent == None:
        break
    if (getpatent[1] == None) or (getpatent[2] == None):
        continue
    else:
        patent_list.append(getpatent[0])
        abstract_title_list.append(getpatent[1] + " " + getpatent[2])


In [49]:
len(abstract_title_list)

822529

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords = nltk.corpus.stopwords.words('english')

In [51]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000000,
                                   stop_words=stopwords,
                                   use_idf=True,
                                   sublinear_tf=True)

In [52]:
%time tfidf_mat = tfidf_vectorizer.fit_transform(abstract_title_list)
print(tfidf_mat.shape)

Wall time: 2min 56s
(822529, 144553)


In [53]:
similarpatents = []

for patent in patents_most_cited:
    ix = patent_list.index(patent)
    similar_patents = cosine_similarity(tfidf_mat[ix], tfidf_mat)
    similar_patents = similar_patents.argsort().flatten()[:-12:-1][1:]
    sim_patent_list = []
    for item in similar_patents:
        sim_patent_list.append(patent_list[item])
    similarpatents.append(sim_patent_list)
    

ValueError: u'D607918' is not in list

In [54]:
len(similarpatents)

2164

In [58]:
similarpatents[2163]

[u'7860964',
 u'9203636',
 u'7945693',
 u'9130756',
 u'9712325',
 u'8397073',
 u'8301748',
 u'8495220',
 u'8060616',
 u'9407681']

In [60]:
len(patents_most_cited)

2171

In [66]:
patents_most_cited2 = patents_most_cited[0:2164]

In [56]:
c.execute('''CREATE TABLE MostCitedPatents (patent_number TEXT PRIMARY KEY);''')

In [62]:
similarpatents[2163]

[u'7860964',
 u'9203636',
 u'7945693',
 u'9130756',
 u'9712325',
 u'8397073',
 u'8301748',
 u'8495220',
 u'8060616',
 u'9407681']

In [69]:
len(patents_most_cited)

2171

In [68]:
c.execute('''CREATE TABLE MostCitedPatents2 (patent_number TEXT PRIMARY KEY);''')

In [70]:
for i in range(len(patents_most_cited)):
    c.execute('INSERT INTO MostCitedPatents VALUES(?)',(patents_most_cited[i],))

In [71]:
for i in range(len(patents_most_cited2)):
    c.execute('INSERT INTO MostCitedPatents2 VALUES(?)',(patents_most_cited2[i],))

In [72]:
c.execute('''CREATE TABLE SimilarPatents (patent_number TEXT PRIMARY KEY,
similarpatents TEXT);''')

In [74]:
print len(similarpatents)
print len(patents_most_cited2)

2164
2164


In [75]:
similarpatents[2150]

[u'7647516',
 u'9292350',
 u'9319335',
 u'9058155',
 u'9703652',
 u'8826272',
 u'9405569',
 u'8370661',
 u'8296590',
 u'8688831']

In [76]:
for i, entry in enumerate(similarpatents):
    c.execute('INSERT INTO SimilarPatents VALUES(?,?)',(patents_most_cited2[i],','.join(entry)))

In [77]:
conn.commit()

In [78]:
patent_list = []
abstract_title_list = []

In [79]:
len(similar_patents)

10